In [13]:
import pandas as pd 
import numpy as np
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from lxml import etree
import csv
import os
from time import sleep
from random import randint

In [14]:
stock_name = ['宝钢股份']

In [5]:
url = 'https://www.baidu.com/s?ie=utf-8&medium=0&rtt=1&bsst=1&rsv_dl=news_t_sk&cl=2&wd=宝钢股份'

In [6]:
headers = {'user-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}

In [40]:
def parseTime(unformatedTime):
    if '分钟' in unformatedTime:
        minute = unformatedTime[:unformatedTime.find('分钟')]
        minute = timedelta(minutes=int(minute))
        return (datetime.now() -
                minute).strftime('%Y-%m-%d %H:%M')
    elif '小时' in unformatedTime:
        hour = unformatedTime[:unformatedTime.find('小时')]
        hour = timedelta(hours=int(hour))
        return (datetime.now() -
                hour).strftime('%Y-%m-%d %H:%M')
    else:
        return unformatedTime


def dealHtml(html):
    results = html.xpath('//div[@class="result-op c-container xpath-log new-pmd"]')

    saveData = []

    for result in results:
        title = result.xpath('.//h3/a')[0]
        title = title.xpath('string(.)').strip()

        summary = result.xpath('.//span[@class="c-font-normal c-color-text"]')[0]
        summary = summary.xpath('string(.)').strip()

        # ./ 是直接下级，.// 是直接/间接下级
#         infos = result.xpath('.//div[@class="news-source"]')[0]
#         source, dateTime = infos.xpath(".//span[last()-1]/text()")[0], \
#                           infos.xpath(".//span[last()]/text()")[0]
        
        try:
            infos = result.xpath('.//div[@class="news-source"]')[0]
            source, dateTime = infos.xpath(".//span[last()-1]/text()")[0], \
                              infos.xpath(".//span[last()]/text()")[0]
        except:
            continue 
        
        dateTime = parseTime(dateTime)

        print('标题', title)
        print('来源', source)
        print('时间', dateTime)
        print('概要', summary)
        print('\n')

        saveData.append({
            'title': title,
            'source': source,
            'time': dateTime,
            'summary': summary
        })
    with open(fileName, 'a+', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        for row in saveData:
            writer.writerow([row['title'], row['source'], row['time'], row['summary']])


#headers = {
#    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36',
#    'Referer': 'https://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&word=%B0%D9%B6%C8%D0%C2%CE%C5&fr=zhidao'
#}
headers = headers
url = 'https://www.baidu.com/s'

params = {
    'ie': 'utf-8',
    'medium': 0,
    # rtt=4 按时间排序 rtt=1 按焦点排序
    'rtt': 1,
    'bsst': 1,
    'rsv_dl': 'news_t_sk',
    'cl': 2,
    'tn': 'news',
    'rsv_bp': 1,
    'oq': '',
    'rsv_btype': 't',
    'f': 8,
}


def doSpider(keyword, sortBy = 'focus'):
    '''
    :param keyword: 搜索关键词
    :param sortBy: 排序规则，可选：focus(按焦点排序），time(按时间排序），默认 focus
    :return:
    '''
    global fileName
    fileName = '{}.csv'.format(keyword)

    if not os.path.exists(fileName):
        with open(fileName, 'w+', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['title', 'source', 'time', 'summary'])

    params['wd'] = keyword
    if sortBy == 'time':
        params['rtt'] = 4

    response = requests.get(url=url, params=params, headers=headers)

    html = etree.HTML(response.text)

    dealHtml(html)

    total = html.xpath('//div[@id="header_top_bar"]/span/text()')[0]

    total = total.replace(',', '')

    total = int(total[7:-1])

    #pageNum = total // 10000
    pageNum = 10

    for page in range(1, pageNum):
        print('第 {} 页\n\n'.format(page))
        headers['Referer'] = response.url
        params['pn'] = page * 10

        response = requests.get(url=url, headers=headers, params=params)

        html = etree.HTML(response.text)

        dealHtml(html)

        sleep(randint(2, 4))
    ...


if __name__ == "__main__":
    doSpider(keyword = '宝钢股份', sortBy='focus')

标题 中远海控、顺丰控股、隆基股份、龙蟒佰利、宝钢股份
来源 腾讯网
时间 前天21:40
概要 宝钢股份、太钢不锈 钢铁行业景气的走势比较强劲,一是因为资金抛弃高估值的抱团股,转向低估值的板块。现在投资钢铁、房地产、银行等低估值板块,虽然不能赚很多钱,但至少不会亏钱。在现在弱势市场中,资金避险情绪比较高,低估值板块比较受...


标题 宝钢股份“智慧制造”战略进一步落地 BSI宝钢标准产品交易价格...
来源 公告解读
时间 5天前
概要 上证报中国证券网讯 粗钢产量问鼎世界第一的中国宝武旗下宝钢股份4月8日对外发布“BSI宝钢标准产品交易价格指数”(下称“宝钢BSI指数”),它围绕碳钢板材领域热轧、普冷标准品的当日价格、供需分析以及相关资讯,为下游各方用户打造了一个...


标题 宝钢股份“超轻型纯电动高安全白车身”全球首发
来源 新民晚报
时间 4天前
概要 图说:“超轻型纯电动高安全白车身”全球首发 采访对象供图 新民晚报讯(记者 叶薇)中国宝武旗下宝钢股份研制的超轻型纯电动高安全白车身“BCB EV”今天首发,向全球用户展示宝钢白车身项目的最新研究成果。 面对近年来新能源汽车的发展浪潮...


标题 宝钢股份5月份国内期货销售价格最高上调500元/吨
来源 同花顺财经
时间 3天前
概要 宝钢股份5月份国内期货销售价格调整,热轧基价上调400元/吨;厚板碳钢厚板基价上调500元/吨;型钢基价上调200元/吨;酸洗基价上调300元/吨;普冷基价上调150元/吨;热镀锌基价不变;电镀锌基价不变;无取向硅钢中低牌号基价不变,高效、...


标题 宝钢股份2021年5月份板材产品价格政策
来源 东方财富网
时间 4天前
概要 经研究决定,宝钢股份碳钢内销价格2021年5月份在2021年4月份基础上调整公告如下(以下除特殊注明外,均为不含税): 一、热轧 基价上调400元/吨。 二、厚板 碳钢厚板基价上调500元/吨。 三、型钢 ...


标题 宝钢股份、太钢不锈人事变动!—中国钢铁新闻网
来源 中国钢铁新闻网
时间 5天前
概要 宝钢股份(600019)4月6日发布公告称,宝山钢铁股份有限公司董事会近日收到该公司副总经理魏成文先生、副总经理胡玉良女士递交的书面辞职报告。魏成文先生、胡玉良女士因工作另有安排辞去该公司副总经理职务。


标题 宝钢股份超轻型

第 5 页


标题 宝钢股份拟回购1.80%至2.25%公司股份
来源 人民资讯
时间 1月19日
概要 上证报中国证券网讯(记者 孔子元)宝钢股份公告,公司拟回购A股股份,用于实施股权激励计划。回购金额不超过40亿元,回购价格不超8.09元/股。回购股份数量不低于4亿股,不超过5亿股,占公司回购前总股本约1.80%-2.25%。公司同日发布业绩...


标题 巴菲特点破A股市场:10年前买入5万元宝钢股份这种低价股持有到现在...
来源 腾讯网
时间 1月2日
概要 2010年1月4日买入5万元宝钢股份,当天开盘价9.7元,持仓为5100股,余额为530元。 第一年,2010年5月24日10股派2元,所得分红为510*2*0.9=918元,当天开盘6.63元,继续买入200股,持仓为5300股,余额为122元。


标题 宝钢股份参股子公司环境违法被罚6.5万元
来源 每日经济新闻
时间 2020年12月18日
概要 每经AI快讯,“A股绿色报告”项目监控到的数据显示,宝钢股份(600019.SH)参股子公司山东宝华耐磨钢有限公司因环境违法行为收到行政处罚。山东宝华耐磨钢有限公司被处以罚款6.5万元。该处罚信息于2020年11月26日被相关监管机构披露。 “A股...


标题 宝钢股份无取向硅钢新产线开工,总投资27.2亿元
来源 界面新闻
时间 2020年12月9日
概要 上海市宝山区政府官方微信称,12月8日,宝钢股份无取向硅钢产品结构优化项目在当地开工。 该项目是目前全球唯一的完全面向新能源汽车行业的高等级无取向硅钢专业生产线,总投资27.2亿元,计划2023年3月建成投产。 建成投产后,宝钢股份硅钢产品...


标题 宝钢股份:公司的发展战略为一种模式、五大能力 简称1+5战略
来源 金融界
时间 2020年11月16日
概要 同花顺(300033)金融研究中心11月16日讯,有投资者向宝钢股份(600019)提问, 身为长期看好公司发展的中小股东,想问下公司在三季度业绩说明会上提到的公司1+5战略具体指的是什么? 公司回答表示,感谢您对公司的关注与支持,公司的发展战略为...


标题 招商银行、宝钢股份等股获北向资金连续5天增持
来源 证券时报
时间 2020年11月6日
概要 据同花顺iFinD统计,近5个交易日(10月30日-11月5日)7